In [1]:
!pip install -q langchain
!pip install -q transfomers sentencepiece accelerate bitsandbytes
!pip install -q python-dotenv
!pip install langchain-google-genai
!pip install google-colab
!pip install langchain langchain-community
!pip install -q unstructured
!pip install nltk
!pip install -q wikipedia
!pip install -q pypdf
!pip install -q tiktoken
!pip install chromadb

ERROR: Could not find a version that satisfies the requirement transfomers (from versions: none)
ERROR: No matching distribution found for transfomers
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 64.0 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 49.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 11.2 MB/s eta 0:00:00
  Prepari

In [35]:
from dotenv import dotenv_values
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage
from langchain_community.chat_models import ChatGooglePalm
import os
import pandas as pd
from langchain.document_loaders import DataFrameLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import WikipediaLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import OnlinePDFLoader
from langchain.document_loaders import DirectoryLoader
from transformers import AutoTokenizer
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain.chains import load_summarize_chain
from langchain.chains.question_answering import load_qa_chain


env_values = dotenv_values("./app.env")

google_api_key = env_values['GOOGLE_API_KEY']

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7,
    google_api_key=google_api_key
)

In [36]:
query1="Nikola Tesla"
query2="Thomas Edison"

docs1=WikipediaLoader(query=query1,load_max_docs=1,doc_content_chars_max=20000).load()
docs2=WikipediaLoader(query=query2,load_max_docs=1,doc_content_chars_max=20000).load()

text_document1=docs1[0].page_content
text_documnet2=docs2[0].page_content
documents=[text_document1,text_documnet2]
metadatas=[{"source":query1},{"source":query2}]

In [37]:
documents[0]

'Nikola Tesla (; Serbian Cyrillic: Никола Тесла [nǐkola têsla]; 10 July 1856 – 7 January 1943) was a Serbian-American engineer, futurist, and inventor. He is known for his contributions to the design of the modern alternating current (AC) electricity supply system.\nBorn and raised in the Austrian Empire, Tesla first studied engineering and physics in the 1870s without receiving a degree. He then gained practical experience in the early 1880s working in telephony and at Continental Edison in the new electric power industry. In 1884 he immigrated to the United States, where he became a naturalized citizen. He worked for a short time at the Edison Machine Works in New York City before he struck out on his own. With the help of partners to finance and market his ideas, Tesla set up laboratories and companies in New York to develop a range of electrical and mechanical devices. His AC induction motor and related polyphase AC patents, licensed by Westinghouse Electric in 1888, earned him a c

In [38]:
from langchain.text_splitter import NLTKTextSplitter
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [39]:
text_splitter = NLTKTextSplitter(chunk_size=300, chunk_overlap=50)
tokens_chunks = text_splitter.create_documents(documents, metadatas=metadatas)
len(tokens_chunks)

177

In [40]:
tokens_chunks[5]

Document(metadata={'source': 'Nikola Tesla'}, page_content='Attempting to develop inventions he could patent and market, Tesla conducted a range of experiments with mechanical oscillators/generators, electrical discharge tubes, and early X-ray imaging.\n\nHe also built a wirelessly controlled boat, one of the first ever exhibited.')

In [41]:
docs_text=[chunk.page_content for chunk in tokens_chunks]
docs_text[5]

'Attempting to develop inventions he could patent and market, Tesla conducted a range of experiments with mechanical oscillators/generators, electrical discharge tubes, and early X-ray imaging.\n\nHe also built a wirelessly controlled boat, one of the first ever exhibited.'

In [42]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"

embedding_llm = SentenceTransformerEmbeddings(model_name=model_name)

In [43]:
save_to_dir = "/content/wiki_chroma_db"

docs_ids = list( range( len(tokens_chunks) ) )
docs_ids = [ str(d) for d in docs_ids ]

vector_db = Chroma.from_documents(
                                tokens_chunks,
                                embedding_llm,
                                persist_directory=save_to_dir,
                                ids=docs_ids
                            )

In [44]:
qna_template = "\n".join([
    "Answer the next questionu using the provided context.",
    "If the answer is not contained in the context, say 'NO ANSWER IS AVAILABLE'",
    "### Context:",
    "{context}",
    "",
    "### Question:",
    "{question}",
    "",
    "### Answer:",
])

qna_prompt = PromptTemplate(
    template=qna_template,
    input_variables=['context', 'question'],
    verbose=True
)

stuff_chain = load_qa_chain(llm, chain_type="stuff", prompt=qna_prompt)

In [45]:
question = "Why did Tesla work as a football player?"

similar_docs = vector_db.similarity_search(question, k=4)

print(len(similar_docs))

4


In [46]:
question1="In which year did Tesla's faher die"
similar_docs1=vector_db.similarity_search(question1,k=4)

In [47]:
answer = stuff_chain(
    {
        "input_documents": similar_docs,
        "question": question
    },
    return_only_outputs=True,
)

answer

{'output_text': 'NO ANSWER IS AVAILABLE'}

In [48]:
answer = stuff_chain(
    {
        "input_documents": similar_docs1,
        "question": question1
    },
    return_only_outputs=True,
)

answer

{'output_text': '1879'}

In [49]:
initial_qna_template = "\n".join([
    "Answer the following question using the provided text only.",
    "If answer is not available. Say 'No answer for this context'",
    "### Context:",
    "{context_str}",
    "",
    "### Question:",
    "{question}",
    "### Answer:",
])

initial_qna_prompt = PromptTemplate(
    template=initial_qna_template,
    input_variables=['context_str', 'question']
)

In [50]:
refine_qna_template = "\n".join([
    "Refine the existing answer, if required, with the following context.",
    "If answer is not available. Say 'No answer for this context'",
    "### Context",
    "{context_str}",
    "",
    "### Existing Answer:",
    "{existing_answer}",
    "",
    "### Question:",
    "{question}",
    "",
    "### Refined Answer:",
])

refine_qna_prompt = PromptTemplate(
    template=refine_qna_template,
    input_variables=['context_str', 'existing_answer', 'question']
)

In [51]:
refine_chain = load_qa_chain(
    llm,
    chain_type="refine",
    question_prompt=initial_qna_prompt,
    refine_prompt=refine_qna_prompt,
    return_intermediate_steps=True,
)

In [52]:
question = "What did Tesla invent?"

similar_docs = vector_db.similarity_search(question, k=2)

print( len(similar_docs) )

2


In [53]:
final_refined_answer = refine_chain({
                                        "input_documents": similar_docs,
                                        "question": question
                                    }, return_only_outputs=True)

final_refined_answer

{'intermediate_steps': ['According to the text, Tesla invented:\n\n*   Mechanical oscillators/generators\n*   Electrical discharge tubes\n*   A wirelessly controlled boat',
  "The existing answer is inaccurate based on the provided context. The context states that Tesla agreed to back an *inventor* financially and handle *his* patents, based on new ideas for electrical equipment, including a thermo-magnetic motor. The context doesn't say Tesla *invented* these things, but rather that he *supported* the inventor.\n\nTherefore, the answer should be:\n\nNo answer for this context"],
 'output_text': "The existing answer is inaccurate based on the provided context. The context states that Tesla agreed to back an *inventor* financially and handle *his* patents, based on new ideas for electrical equipment, including a thermo-magnetic motor. The context doesn't say Tesla *invented* these things, but rather that he *supported* the inventor.\n\nTherefore, the answer should be:\n\nNo answer for t